<a href="https://colab.research.google.com/github/krjohnn/LU-VTL-25m/blob/main/MD3/DatZM037_VTL_MD3_Benchmarks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Install libraries
# We need 'accelerate' and 'bitsandbytes' to run models efficiently
!pip install -q transformers torch accelerate huggingface_hub google-generativeai bitsandbytes

import json
import torch
import time
import gc # Garbage collector for memory management
from transformers import pipeline
from huggingface_hub import login
from google.colab import userdata
import google.generativeai as genai

In [ ]:
# =========================================================
# PART 1: AUTHENTICATION
# =========================================================

# Hugging Face Login
try:
    hf_token = userdata.get('HuggingFace')
    login(token=hf_token)
    print("Logged in to Hugging Face successfully.")
except Exception as e:
    print("Error logging in. Make sure 'HuggingFace' secret is set in Colab.")

# Google Gemini Login
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Google API configured successfully.")
except Exception as e:
    print("Error: Please set 'GOOGLE_API_KEY' in Colab Secrets.")

# Check GPU
if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not detected. Please enable T4 GPU in Runtime settings.")

In [ ]:
# =========================================================
# PART 2: FULL DATASET (100 QUESTIONS)
# =========================================================
dataset = [
  { "id": 1, "jautajums": "Kura ir garākā upe, kas plūst tikai Latvijas teritorijā?", "varianti": ["Daugava", "Gauja", "Lielupe", "Venta"], "pareiza_atbilde": "Gauja" },
  { "id": 2, "jautajums": "Kurš ir augstākais paugurs Latvijā?", "varianti": ["Gaiziņkalns", "Sirdskalns", "Lielais Liepukalns", "Mākoņkalns"], "pareiza_atbilde": "Gaiziņkalns" },
  { "id": 3, "jautajums": "Kura pilsēta tiek dēvēta par vēju pilsētu?", "varianti": ["Ventspils", "Liepāja", "Rīga", "Jūrmala"], "pareiza_atbilde": "Liepāja" },
  { "id": 4, "jautajums": "Kurš ir dziļākais ezers Latvijā?", "varianti": ["Lubāns", "Rāznas ezers", "Drīdzis", "Burtnieks"], "pareiza_atbilde": "Drīdzis" },
  { "id": 5, "jautajums": "Kurā Latvijas novadā atrodas Tērvetes dabas parks?", "varianti": ["Vidzemē", "Latgalē", "Kurzemē", "Zemgalē"], "pareiza_atbilde": "Zemgalē" },
  { "id": 6, "jautajums": "Kas ir Latvijas nacionālais putns?", "varianti": ["Baltā cielava", "Melnais stārķis", "Zilzīlīte", "Dzērve"], "pareiza_atbilde": "Baltā cielava" },
  { "id": 7, "jautajums": "Kura ir platākā upe Latvijā?", "varianti": ["Gauja", "Venta", "Daugava", "Lielupe"], "pareiza_atbilde": "Venta" },
  { "id": 8, "jautajums": "Kurā gadā tika dibināts Gaujas Nacionālais parks?", "varianti": ["1973", "1991", "1955", "2004"], "pareiza_atbilde": "1973" },
  { "id": 9, "jautajums": "Kura ir Latvijas lielākā sala (jūrā vai līcī)?", "varianti": ["Doles sala", "Moricsala", "Latvijai nav jūras salu", "Lucavsala"], "pareiza_atbilde": "Latvijai nav jūras salu" },
  { "id": 10, "jautajums": "Kurš ir lielākais ezers Latvijā pēc platības?", "varianti": ["Rāznas ezers", "Lubāns", "Engures ezers", "Burtnieks"], "pareiza_atbilde": "Lubāns" },
  { "id": 11, "jautajums": "Kura pilsēta atrodas pie Ventas rumbas?", "varianti": ["Ventspils", "Kuldīga", "Sabile", "Skrunda"], "pareiza_atbilde": "Kuldīga" },
  { "id": 12, "jautajums": "Cik kultūrvēsturiskie novadi veido Latviju (ieskaitot Sēliju)?", "varianti": ["3", "4", "5", "6"], "pareiza_atbilde": "5" },
  { "id": 13, "jautajums": "Kurš ir Latvijas nacionālais koks?", "varianti": ["Ozols", "Liepa", "Bērzs", "Priede"], "pareiza_atbilde": "Ozols" },
  { "id": 14, "jautajums": "Kura pilsēta ir Vidzemes centrālā pilsēta?", "varianti": ["Cēsis", "Valmiera", "Sigulda", "Smiltene"], "pareiza_atbilde": "Valmiera" },
  { "id": 15, "jautajums": "Kur atrodas Kolkasrags?", "varianti": ["Kurzemes ziemeļos", "Vidzemes piekrastē", "Rīgas centrā", "Latgales pierobežā"], "pareiza_atbilde": "Kurzemes ziemeļos" },
  { "id": 16, "jautajums": "Kā sauc ūdenskritumu, kas atrodas uz Ventas upes?", "varianti": ["Rumbas kritums", "Ventas rumba", "Abavas rumba", "Ivandes rumba"], "pareiza_atbilde": "Ventas rumba" },
  { "id": 17, "jautajums": "Kurš ir Latvijas nacionālais zieds?", "varianti": ["Rudzupuķe", "Pīene", "Magone", "Jasmīns"], "pareiza_atbilde": "Rudzupuķe" },
  { "id": 18, "jautajums": "Kura pilsēta ir pazīstama ar saviem pilsdrupām un Livonijas ordeņa pili?", "varianti": ["Jelgava", "Cēsis", "Ogre", "Salaspils"], "pareiza_atbilde": "Cēsis" },
  { "id": 19, "jautajums": "Kurā Latvijas pilsētā atrodas Latgales vēstniecība GORS?", "varianti": ["Daugavpilī", "Rēzeknē", "Krāslavā", "Ludzā"], "pareiza_atbilde": "Rēzeknē" },
  { "id": 20, "jautajums": "Cik gara ir Latvijas jūras robeža (aptuveni)?", "varianti": ["300 km", "498 km", "250 km", "600 km"], "pareiza_atbilde": "498 km" },
  { "id": 21, "jautajums": "Kurš ir Latvijas lielākais laukakmens?", "varianti": ["Nīcgales Lielais akmens", "Velna akmens", "Lauču akmens", "Radžu akmens"], "pareiza_atbilde": "Nīcgales Lielais akmens" },
  { "id": 22, "jautajums": "Kurš ir lielākais purvs Latvijā?", "varianti": ["Ķemeru tīrelis", "Teiču purvs", "Cenas tīrelis", "Sedas purvs"], "pareiza_atbilde": "Teiču purvs" },
  { "id": 23, "jautajums": "Kurā pilsētā atrodas Latvijā garākais tilts pāri upei?", "varianti": ["Rīgā", "Daugavpilī", "Jelgavā", "Jēkabpilī"], "pareiza_atbilde": "Rīgā" },
  { "id": 24, "jautajums": "Kura upe ietek Rīgas jūras līcī pie Rīgas?", "varianti": ["Lielupe", "Daugava", "Gauja", "Salaca"], "pareiza_atbilde": "Daugava" },
  { "id": 25, "jautajums": "Kura no šīm pilsētām atrodas Daugavas krastos?", "varianti": ["Jēkabpils", "Jelgava", "Valmiera", "Tukums"], "pareiza_atbilde": "Jēkabpils" },
  { "id": 26, "jautajums": "Kur atrodas Zvārtes iezis?", "varianti": ["Pie Gaujas", "Pie Amatas", "Pie Ventas", "Pie Daugavas"], "pareiza_atbilde": "Pie Amatas" },
  { "id": 27, "jautajums": "Kura ir Latvijas galvaspilsēta?", "varianti": ["Rīga", "Jelgava", "Liepāja", "Daugavpils"], "pareiza_atbilde": "Rīga" },
  { "id": 28, "jautajums": "Kurā Latvijas daļā atrodas 'Zilo ezeru zeme'?", "varianti": ["Kurzemē", "Vidzemē", "Latgalē", "Zemgalē"], "pareiza_atbilde": "Latgalē" },
  { "id": 29, "jautajums": "Cik zvaigznes ir Brīvības pieminekļa smailē?", "varianti": ["1", "3", "5", "4"], "pareiza_atbilde": "3" },
  { "id": 30, "jautajums": "Kuru pilsētu dēvē par 'Vidzemes Šveici'?", "varianti": ["Siguldu", "Cēsis", "Madonu", "Ērgļus"], "pareiza_atbilde": "Siguldu" },
  { "id": 31, "jautajums": "Kurš ir vecākais Nacionālais parks Latvijā?", "varianti": ["Ķemeru", "Rāznas", "Gaujas", "Slīteres"], "pareiza_atbilde": "Gaujas" },
  { "id": 32, "jautajums": "Kur atrodas Aglonas bazilika?", "varianti": ["Zemgalē", "Kurzemē", "Latgalē", "Vidzemē"], "pareiza_atbilde": "Latgalē" },
  { "id": 33, "jautajums": "Kura ir Latvijas tālākā ziemeļu punkta atrašanās vieta?", "varianti": ["Ipiķi", "Ainaži", "Kolka", "Rūjiena"], "pareiza_atbilde": "Ipiķi" },
  { "id": 34, "jautajums": "Kas ir Latvijas nacionālais kukainis?", "varianti": ["Divpunktu mārīte", "Maijvabole", "Kamene", "Spāre"], "pareiza_atbilde": "Divpunktu mārīte" },
  { "id": 35, "jautajums": "Kurā pilsētā atrodas Karosta?", "varianti": ["Ventspilī", "Liepājā", "Rīgā", "Daugavpilī"], "pareiza_atbilde": "Liepājā" },
  { "id": 36, "jautajums": "Kurš ezers ir visbagātākais ar salām?", "varianti": ["Ežezers", "Burtnieks", "Rāznas ezers", "Usmas ezers"], "pareiza_atbilde": "Ežezers" },
  { "id": 37, "jautajums": "Kura upe ir robežupe starp Latviju un Igauniju?", "varianti": ["Gauja", "Salaca", "Melnupe", "Pededze"], "pareiza_atbilde": "Melnupe" },
  { "id": 38, "jautajums": "Kur atrodas Gūtmaņa ala?", "varianti": ["Līgatnē", "Siguldā", "Kuldīgā", "Mazsalacā"], "pareiza_atbilde": "Siguldā" },
  { "id": 39, "jautajums": "Kura ir otra lielākā pilsēta Latvijā pēc iedzīvotāju skaita?", "varianti": ["Liepāja", "Jelgava", "Daugavpils", "Jūrmala"], "pareiza_atbilde": "Daugavpils" },
  { "id": 40, "jautajums": "Kurš ir augstākais skatu tornis Latvijā (bez TV torņiem)?", "varianti": ["Lielā Liepukalna tornis", "Gaiziņkalna tornis", "Zilākalna tornis", "Tērvetes tornis"], "pareiza_atbilde": "Lielā Liepukalna tornis" },
  { "id": 41, "jautajums": "Ar kurām valstīm Latvijai ir sauszemes robeža?", "varianti": ["Igauniju, Lietuvu, Krieviju, Baltkrieviju", "Igauniju, Lietuvu, Poliju, Krieviju", "Igauniju, Lietuvu, Zviedriju", "Lietuvu, Krieviju, Baltkrieviju, Ukrainu"], "pareiza_atbilde": "Igauniju, Lietuvu, Krieviju, Baltkrieviju" },
  { "id": 42, "jautajums": "Kur atrodas Rundāles pils?", "varianti": ["Bauskas novadā", "Jelgavas novadā", "Tukuma novadā", "Kuldīgas novadā"], "pareiza_atbilde": "Bauskas novadā" },
  { "id": 43, "jautajums": "Kā sauc lielāko smilšakmens atsegumu Latvijā?", "varianti": ["Sietiņiezis", "Zvārtes iezis", "Ērgļu klintis", "Baltā kāpa"], "pareiza_atbilde": "Baltā kāpa" },
  { "id": 44, "jautajums": "Kurā pilsētā notiek ikgadējais 'Positivus' festivāls (līdz 2022.g)?", "varianti": ["Salacgrīvā", "Saulkrastos", "Liepājā", "Siguldā"], "pareiza_atbilde": "Salacgrīvā" },
  { "id": 45, "jautajums": "Kura upe tek cauri Valmierai?", "varianti": ["Daugava", "Gauja", "Salaca", "Ogre"], "pareiza_atbilde": "Gauja" },
  { "id": 46, "jautajums": "Kāda ir Latvijas valsts valūta?", "varianti": ["Lats", "Eiro", "Dolārs", "Kronas"], "pareiza_atbilde": "Eiro" },
  { "id": 47, "jautajums": "Kurš ir mazākais Nacionālais parks Latvijā?", "varianti": ["Slīteres", "Ķemeru", "Rāznas", "Gaujas"], "pareiza_atbilde": "Slīteres" },
  { "id": 48, "jautajums": "Kura pilsēta ir slavena ar savu Brīvdabas muzeju?", "varianti": ["Rīga", "Ogre", "Talsi", "Saldus"], "pareiza_atbilde": "Rīga" },
  { "id": 49, "jautajums": "Kura upe ir pazīstama ar saviem līkumiem (meandriem) pie Krāslavas?", "varianti": ["Daugava", "Gauja", "Venta", "Aiviekste"], "pareiza_atbilde": "Daugava" },
  { "id": 50, "jautajums": "Kurā pilsētā atrodas 'Laimas' pulkstenis?", "varianti": ["Jelgavā", "Rīgā", "Siguldā", "Cēsīs"], "pareiza_atbilde": "Rīgā" },
  { "id": 51, "jautajums": "Kas attēlots Latvijas ģerboņa apakšā?", "varianti": ["Ozola zari un sarkanbaltsarkana lente", "Liepas zari", "Rudzu vārpas", "Jūras viļņi"], "pareiza_atbilde": "Ozola zari un sarkanbaltsarkana lente" },
  { "id": 52, "jautajums": "Kur atrodas Pokaiņu mežs?", "varianti": ["Pie Dobeles", "Pie Talsiem", "Pie Madonas", "Pie Valkas"], "pareiza_atbilde": "Pie Dobeles" },
  { "id": 53, "jautajums": "Kurā gadā Latvija pievienojās Eiropas Savienībai?", "varianti": ["2004", "1991", "2014", "1999"], "pareiza_atbilde": "2004" },
  { "id": 54, "jautajums": "Kura ir Latvijas tālākā dienvidu punkta atrašanās vieta?", "varianti": ["Demene", "Skaistkalne", "Nereta", "Eleja"], "pareiza_atbilde": "Demene" },
  { "id": 55, "jautajums": "Kurš dabas objekts ir attēlots uz 5 eiro banknotes (Latvijas versijā)?", "varianti": ["Nav specifiska dabas objekta", "Gaiziņkalns", "Daugavas loki", "Ozols"], "pareiza_atbilde": "Nav specifiska dabas objekta" },
  { "id": 56, "jautajums": "Kurā pilsētā atrodas Livonijas ordeņa pils drupas un jauna pils, kas celta klasicisma stilā?", "varianti": ["Siguldā", "Cēsīs", "Bauskā", "Valmierā"], "pareiza_atbilde": "Bauskā" },
  { "id": 57, "jautajums": "Kura upe ietek Baltijas jūrā pie Pāvilostas?", "varianti": ["Saka", "Durbe", "Tebra", "Užava"], "pareiza_atbilde": "Saka" },
  { "id": 58, "jautajums": "Kā sauc lielāko salu Daugavas upē?", "varianti": ["Doles sala", "Saknas sala", "Zaķusala", "Lucavsala"], "pareiza_atbilde": "Doles sala" },
  { "id": 59, "jautajums": "Kurš ir Latvijas bagātākais ezers ar zivju sugām?", "varianti": ["Alūksnes ezers", "Rāznas ezers", "Burtnieks", "Drīdzis"], "pareiza_atbilde": "Alūksnes ezers" },
  { "id": 60, "jautajums": "Kura ir Latvijas garākā upe (ieskaitot tecējumu ārpus Latvijas)?", "varianti": ["Daugava", "Gauja", "Lielupe", "Venta"], "pareiza_atbilde": "Daugava" },
  { "id": 61, "jautajums": "Kurā novadā atrodas Turaidas muzejrezervāts?", "varianti": ["Siguldas novadā", "Cēsu novadā", "Saulkrastu novadā", "Ropažu novadā"], "pareiza_atbilde": "Siguldas novadā" },
  { "id": 62, "jautajums": "Kas ir 'Staburags' (vēsturiski)?", "varianti": ["Šūnakmens klints", "Liels dižozols", "Senlatviešu pils", "Ūdenskritums"], "pareiza_atbilde": "Šūnakmens klints" },
  { "id": 63, "jautajums": "Kura pilsēta atrodas pie Abavas rumbas?", "varianti": ["Sabile", "Kandava", "Talsi", "Kuldīga"], "pareiza_atbilde": "Sabile" },
  { "id": 64, "jautajums": "Kas ir 'Velnalas' klintis?", "varianti": ["Smilšakmens atsegums", "Kaļķakmens klintis", "Granīta bluķi", "Māla krasti"], "pareiza_atbilde": "Smilšakmens atsegums" },
  { "id": 65, "jautajums": "Kurā Latvijas pilsētā ir visvairāk tiltu?", "varianti": ["Rīgā", "Liepājā", "Daugavpilī", "Jelgavā"], "pareiza_atbilde": "Rīgā" },
  { "id": 66, "jautajums": "Kurš ir populārākais slēpošanas kalns Vidzemē?", "varianti": ["Žagarkalns", "Milzkalns", "Lemberga hūte", "Kamparkalns"], "pareiza_atbilde": "Žagarkalns" },
  { "id": 67, "jautajums": "Kas ir Latvijas nacionālais dārgakmens?", "varianti": ["Dzintars", "Kvarcs", "Ahāts", "Dimants"], "pareiza_atbilde": "Dzintars" },
  { "id": 68, "jautajums": "Kura ir Latvijas bagātākā ogošanas vieta (purvs)?", "varianti": ["Teiču purvs", "Lielais Ķemeru tīrelis", "Sedas purvs", "Cenas tīrelis"], "pareiza_atbilde": "Teiču purvs" },
  { "id": 69, "jautajums": "Kurš ir augstākais koka skatu tornis Latvijā?", "varianti": ["Lielā Liepukalna tornis", "Tērvetes tornis", "Zilākalna tornis", "Ložmetējkalna tornis"], "pareiza_atbilde": "Lielā Liepukalna tornis" },
  { "id": 70, "jautajums": "Kura upe ir pazīstama kā lašupe?", "varianti": ["Salaca", "Lielupe", "Mēmele", "Ogre"], "pareiza_atbilde": "Salaca" },
  { "id": 71, "jautajums": "Kurā pilsētā atrodas Daugavpils cietoksnis?", "varianti": ["Daugavpilī", "Rēzeknē", "Krāslavā", "Jēkabpilī"], "pareiza_atbilde": "Daugavpilī" },
  { "id": 72, "jautajums": "Kur atrodas Ziemeļu forti?", "varianti": ["Liepājā", "Ventspilī", "Rīgā", "Salacgrīvā"], "pareiza_atbilde": "Liepājā" },
  { "id": 73, "jautajums": "Kā sauc ūdenskritumu Kuldīgā?", "varianti": ["Ventas rumba", "Abavas rumba", "Ivandes rumba", "Alekšupītes ūdenskritums"], "pareiza_atbilde": "Ventas rumba" },
  { "id": 74, "jautajums": "Kurš ir lielākais brīvdabas muzejs Latvijā?", "varianti": ["Latvijas Etnogrāfiskais brīvdabas muzejs", "Pastariņa muzejs", "Vikingu ciems", "Slutišķu sādža"], "pareiza_atbilde": "Latvijas Etnogrāfiskais brīvdabas muzejs" },
  { "id": 75, "jautajums": "Kurā gadā Latvija pasludināja neatkarību?", "varianti": ["1918", "1905", "1920", "1990"], "pareiza_atbilde": "1918" },
  { "id": 76, "jautajums": "Kas ir 'Lielie kapi'?", "varianti": ["Parks un memoriāls Rīgā", "Senkapi Kurzemē", "Karavīru kapi Daugavpilī", "Meža kapi Jūrmalā"], "pareiza_atbilde": "Parks un memoriāls Rīgā" },
  { "id": 77, "jautajums": "Kura pilsēta atrodas vistālāk uz austrumiem?", "varianti": ["Zilupe", "Ludza", "Krāslava", "Balvi"], "pareiza_atbilde": "Zilupe" },
  { "id": 78, "jautajums": "Kas ir 'Gaismas pils'?", "varianti": ["Latvijas Nacionālā bibliotēka", "Rīgas pils", "Melngalvju nams", "Zinātņu akadēmija"], "pareiza_atbilde": "Latvijas Nacionālā bibliotēka" },
  { "id": 79, "jautajums": "Kura upe tek cauri Jelgavai?", "varianti": ["Lielupe", "Venta", "Daugava", "Gauja"], "pareiza_atbilde": "Lielupe" },
  { "id": 80, "jautajums": "Kurā pilsētā notiek Operas svētki (brīvdabas)?", "varianti": ["Siguldā", "Cēsīs", "Rīgā", "Jūrmalā"], "pareiza_atbilde": "Siguldā" },
  { "id": 81, "jautajums": "Kas ir 'Mūra tilts' Rīgā?", "varianti": ["Akmens tilts", "Vanšu tilts", "Dienvidu tilts", "Salu tilts"], "pareiza_atbilde": "Akmens tilts" },
  { "id": 82, "jautajums": "Kur atrodas Minhauzena muzejs?", "varianti": ["Duntē", "Salacgrīvā", "Saulkrastos", "Limbažos"], "pareiza_atbilde": "Duntē" },
  { "id": 83, "jautajums": "Kura ir Latvijas seklākā osta?", "varianti": ["Engures osta", "Rīgas osta", "Ventspils osta", "Liepājas osta"], "pareiza_atbilde": "Engures osta" },
  { "id": 84, "jautajums": "Kā sauc lielāko ezeru Latgalē?", "varianti": ["Rāznas ezers", "Lubāns", "Rušons", "Sīvers"], "pareiza_atbilde": "Rāznas ezers" },
  { "id": 85, "jautajums": "Kurš ir augstākais TV tornis Eiropas Savienībā?", "varianti": ["Zaķusalas TV tornis", "Berlīnes tornis", "Viļņas tornis", "Tallinas tornis"], "pareiza_atbilde": "Zaķusalas TV tornis" },
  { "id": 86, "jautajums": "Kurā pilsētā ražo 'Kārums' sieriņus?", "varianti": ["Rīgā", "Valmierā", "Tukumā", "Jēkabpilī"], "pareiza_atbilde": "Rīgā" },
  { "id": 87, "jautajums": "Kur atrodas Latvijas Universitātes galvenā ēka?", "varianti": ["Rīgā, Raiņa bulvārī", "Jelgavā", "Pārdaugavā", "Vecrīgā"], "pareiza_atbilde": "Rīgā, Raiņa bulvārī" },
  { "id": 88, "jautajums": "Kas ir 'Suitu sievas'?", "varianti": ["Etnogrāfisks ansamblis", "Teātra trupa", "Deju kolektīvs", "Kora grupa"], "pareiza_atbilde": "Etnogrāfisks ansamblis" },
  { "id": 89, "jautajums": "Kurā novadā ir Lībiešu krasts?", "varianti": ["Talsu un Ventspils novadā", "Liepājas novadā", "Limbažu novadā", "Saulkrastu novadā"], "pareiza_atbilde": "Talsu un Ventspils novadā" },
  { "id": 90, "jautajums": "Kas ir 'Cūkmens'?", "varianti": ["AS 'Latvijas valsts meži' tēls", "Multfilmu varonis", "Teātra personāžs", "Folkloras tēls"], "pareiza_atbilde": "AS 'Latvijas valsts meži' tēls" },
  { "id": 91, "jautajums": "Kurā pilsētā atrodas teātris 'Jūras vārti'?", "varianti": ["Ventspilī", "Liepājā", "Jūrmalā", "Rīgā"], "pareiza_atbilde": "Ventspilī" },
  { "id": 92, "jautajums": "Kura upe ir Lielupes pieteka?", "varianti": ["Mēmele", "Gauja", "Venta", "Abava"], "pareiza_atbilde": "Mēmele" },
  { "id": 93, "jautajums": "Kur atrodas 'Latvijas naftas tranzīta' terminālis?", "varianti": ["Ventspilī", "Rīgā", "Liepājā", "Salacgrīvā"], "pareiza_atbilde": "Ventspilī" },
  { "id": 94, "jautajums": "Kā sauc lielāko purvu rezervātu?", "varianti": ["Teiču dabas rezervāts", "Grīņu rezervāts", "Krustkalnu rezervāts", "Moricsalas rezervāts"], "pareiza_atbilde": "Teiču dabas rezervāts" },
  { "id": 95, "jautajums": "Kura pilsēta ir slavena ar savu ceriņu dārzu?", "varianti": ["Dobele", "Jelgava", "Sigulda", "Bauska"], "pareiza_atbilde": "Dobele" },
  { "id": 96, "jautajums": "Kas ir 'Nāves sala'?", "varianti": ["Pussala Daugavā", "Sala Baltijas jūrā", "Sala Usmas ezerā", "Kalns Vidzemē"], "pareiza_atbilde": "Pussala Daugavā" },
  { "id": 97, "jautajums": "Kurā pilsētā atrodas Šlokenbekas muiža?", "varianti": ["Engures novadā (Milzkalnē)", "Tukumā", "Kandavā", "Talsos"], "pareiza_atbilde": "Engures novadā (Milzkalnē)" },
  { "id": 98, "jautajums": "Kura ir vismazāk apdzīvotā pilsēta Latvijā?", "varianti": ["Durbe", "Ainaži", "Subate", "Piltene"], "pareiza_atbilde": "Durbe" },
  { "id": 99, "jautajums": "Kurā gadā tika uzbūvēts Rīgas HES?", "varianti": ["1974", "1960", "1985", "1939"], "pareiza_atbilde": "1974" },
  { "id": 100, "jautajums": "Kā sauc Latvijas valsts himnu?", "varianti": ["Dievs, svētī Latviju!", "Saule, Pērkons, Daugava", "Nevis slinkojot un pūstot", "Pūt, vējiņi"], "pareiza_atbilde": "Dievs, svētī Latviju!" }
]

In [ ]:
# =========================================================
# FIXED FUNCTION: ROBUST CHAT HANDLING
# =========================================================
def ask_local_model(pipe, question, options):
    options_str = "\n".join([f"- {opt}" for opt in options])

    # 1. English Instructions "Sandwich"
    # This forces the model to understand the TASK, even if data is Latvian.
    messages = [
        {
            "role": "user",
            "content": f"""You are a helpful AI assistant.
Your task is to answer the multiple-choice question below.
Select ONLY the correct option text from the list.

Question (in Latvian): {question}
Options:
{options_str}

Output ONLY the text of the correct option. Do not explain."""
        }
    ]

    # 2. Apply Template
    prompt = pipe.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # 3. Generate (No temperature needed for do_sample=False)
    outputs = pipe(
        prompt,
        max_new_tokens=50,
        do_sample=False,
        return_full_text=False
    )

    answer = outputs[0]["generated_text"].strip()
    return answer

In [ ]:
# =========================================================
# PART 3: GEMMA-2B (LOCAL - BASELINE)
# =========================================================
print("\n--- Model 1: Gemma-2b (Local) ---")

model_id = "google/gemma-2b-it"
pipe = pipeline(
    "text-generation",
    model=model_id,
    dtype=torch.float16,
    device_map="auto"
)

gemma_correct = 0
results = []

print("Running Gemma-2b...")
for item in dataset:
    ans = ask_local_model(pipe, item["jautajums"], item["varianti"])
    is_correct = item["pareiza_atbilde"].lower() in ans.lower()
    if is_correct: gemma_correct += 1
    results.append({"id": item["id"], "model": "Gemma-2b", "correct": is_correct})

gemma_acc = (gemma_correct / len(dataset)) * 100
print(f"Gemma-2b Accuracy: {gemma_acc:.2f}%")

# Cleanup to free VRAM for next model
del pipe
gc.collect()
torch.cuda.empty_cache()
print("Memory cleared.")

In [ ]:
# =========================================================
# PART 4: PHI-3.5-MINI (LOCAL - CHALLENGER)
# =========================================================
print("\n--- Model 2: Phi-3.5-mini (Local Challenger) ---")
# Phi-3.5 is small (3.8B) but extremely capable. A great comparison model.

model_id_2 = "microsoft/Phi-3.5-mini-instruct"
pipe_2 = pipeline(
    "text-generation",
    model=model_id_2,
    dtype=torch.float16,
    device_map="auto"
)

phi_correct = 0
results_phi = []

print("Running Phi-3.5-mini (Fixed)...")

# Limit to first 20 for a quick check, or remove [:20] for full run
for item in dataset:
    ans = ask_local_model(pipe_2, item["jautajums"], item["varianti"])

    # Check correctness
    is_correct = item["pareiza_atbilde"].lower() in ans.lower()

    if is_correct:
        phi_correct += 1

    results_phi.append({
        "id": item["id"],
        "answer": ans,
        "correct": is_correct
    })

phi_acc = (phi_correct / len(dataset)) * 100
print(f"Phi-3.5 Accuracy (Fixed): {phi_acc:.2f}%")

# Cleanup
import gc
del pipe_2
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# =========================================================
# PART 5: GEMINI-2.5 (CLOUD - ORACLE)
# =========================================================
print("\n--- Model 3: Gemini-2.5-flash-lite (Cloud) ---")

gemini_model = genai.GenerativeModel('gemini-2.5-flash-lite')

def ask_gemini_safe(question, options):
    options_str = "\n".join([f"- {opt}" for opt in options])
    prompt = f"""
    You are a quiz master. Answer the following question about Latvia.
    Question: {question}
    Options:
    {options_str}

    Output ONLY the correct option text. Do not add explanations.
    Correct answer:
    """

    for attempt in range(3):
        try:
            time.sleep(4) # Rate limit pause
            response = gemini_model.generate_content(prompt)
            return response.text.strip()
        except Exception as e:
            print(f"Error ({e}). Waiting 15s...")
            time.sleep(15)
    return "Error"

gemini_correct = 0
print("Running Gemini...")

for item in dataset:
    ans = ask_gemini_safe(item["jautajums"], item["varianti"])
    is_correct = item["pareiza_atbilde"].lower() in ans.lower()
    if is_correct: gemini_correct += 1
    results.append({"id": item["id"], "model": "Gemini-2.5", "correct": is_correct})

gemini_acc = (gemini_correct / len(dataset)) * 100

In [ ]:
# =========================================================
# FINAL RESULTS
# =========================================================
print(f"\n--- FINAL BENCHMARK RESULTS ---")
print(f"1. Gemma-2b (Base):          {gemma_acc:.2f}%")
print(f"2. Phi-3.5-mini (Challenger): {phi_acc:.2f}%")
print(f"3. Gemini-2.5-flash (Cloud):  {gemini_acc:.2f}%")

with open("benchmark_results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)